In [8]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [9]:
import os
print(os.getcwd())

/Users/antheaang/Desktop/dsa4263 /DSA4263-automated-fraud/notebooks


In [10]:
# Load KNN CSV
knn_combined_df = pd.read_csv('../data/processed/knn_combined_df.csv')
knn_combined_df.head()

,entropy,clicks_count,durations,click_frequency,speed_cv,hover_frequency,avg_hover_duration,acceleration_mean,acceleration_std,acceleration_skewness,acceleration_pos_neg_ratio,direction_change_frequency,idle_active_ratio,time_afternoon,time_evening,time_morning,time_night,time_wee_hours,KNN Prediction,True Label
0,-0.655538,-0.156923,-1.429112,1.397659,-0.761518,0.924815,-0.005883,0.902351,-1.024863,0.123380,1.224996,0.744609,-0.857226,1.351581,-0.456896,-0.744387,-0.420772,-0.118846,bot,bot
1,0.698889,-0.266259,-1.429112,0.872302,1.452376,-0.934238,-0.367982,0.069175,1.637779,-0.184886,-0.657948,-0.656709,-0.857226,-0.739874,-0.456896,-0.744387,2.376583,-0.118846,human,human
2,-1.385360,-0.320927,0.700444,-0.465040,-0.688507,1.051944,-0.494030,0.702442,-1.060693,2.558336,0.417644,0.460473,1.634258,-0.739874,-0.456896,1.343387,-0.420772,-0.118846,bot,bot
3,-0.541610,-0.238925,-1.429111,0.338231,-0.303115,-0.522901,0.048134,0.786227,-0.666761,-1.411023,0.582453,-0.638967,-0.857226,1.351581,-0.456896,-0.744387,-0.420772,-0.118846,bot,bot
4,-1.716048,-0.320927,0.700444,-0.465040,-1.063119,1.107620,-0.495586,0.820646,-1.173348,1.876954,0.236468,-0.109728,1.608131,-0.739874,-0.456896,1.343387,-0.420772,-0.118846,bot,bot


In [11]:
le = LabelEncoder()

# Encode the KNN predictions
knn_combined_df['KNN Prediction Encoded'] = le.fit_transform(knn_combined_df['KNN Prediction'])

le.fit_transform(knn_combined_df['KNN Prediction'])


array([0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1])

In [12]:
# Prepare features and labels
# Drop the original KNN Prediction (string) and True Label (target)
X_features = knn_combined_df.drop(columns=['KNN Prediction', 'True Label'])
y_true = knn_combined_df['True Label']


In [13]:
# Initialize Decision Tree
dtree = DecisionTreeClassifier(max_depth=3, min_samples_leaf=2, random_state=42)
dtree.fit(X_features, y_true)

# 5. Predict and Evaluate
y_pred_tree = dtree.predict(X_features)


In [14]:
y_pred_tree = dtree.predict(X_features)
y_pred_tree 

array(['bot', 'human', 'bot', 'bot', 'bot', 'bot', 'human', 'bot', 'bot',
       'bot', 'bot', 'bot', 'human', 'bot', 'bot', 'bot', 'bot', 'bot',
       'bot', 'bot', 'bot', 'bot', 'bot', 'bot', 'bot', 'human', 'human',
       'bot', 'bot', 'bot', 'bot', 'human', 'human', 'human', 'bot',
       'bot', 'bot', 'bot', 'human', 'bot', 'bot', 'bot', 'bot', 'bot',
       'bot', 'bot', 'bot', 'human', 'bot', 'bot', 'human', 'bot', 'bot',
       'bot', 'bot', 'bot', 'bot', 'bot', 'bot', 'human', 'human', 'bot',
       'bot', 'bot', 'human', 'bot', 'bot', 'bot', 'bot', 'bot', 'bot',
       'bot', 'bot', 'bot', 'bot', 'bot', 'bot', 'bot', 'human', 'bot',
       'human', 'bot', 'bot', 'bot', 'bot', 'bot', 'bot', 'bot', 'bot',
       'human'], dtype=object)

In [15]:
print("Decision Tree Ensemble Results:")
print(f"Accuracy: {accuracy_score(y_true, y_pred_tree):.2f}")

print("\nClassification Report:")
print(classification_report(y_true, y_pred_tree))

Decision Tree Ensemble Results:
Accuracy: 1.00

Classification Report:
              precision    recall  f1-score   support

         bot       1.00      1.00      1.00        73
       human       1.00      1.00      1.00        17

    accuracy                           1.00        90
   macro avg       1.00      1.00      1.00        90
weighted avg       1.00      1.00      1.00        90



In [16]:
final_results_df = knn_combined_df.copy()

final_results_df['Decision Tree Prediction'] = y_pred_tree

# Rearrange columns 
final_results_df = final_results_df[[
    # features
    *[col for col in knn_combined_df.columns if col not in ['KNN Prediction', 'True Label', 'KNN Prediction Encoded']],
    # model predictions
    'KNN Prediction', 
    'Decision Tree Prediction',
    'True Label'
]]

final_results_df = final_results_df[['KNN Prediction', 'Decision Tree Prediction', 'True Label']]

# View
final_results_df.head()

,KNN Prediction,Decision Tree Prediction,True Label
0,bot,bot,bot
1,human,human,human
2,bot,bot,bot
3,bot,bot,bot
4,bot,bot,bot
